In [10]:
from PyPDF2 import PdfReader
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import os
from pathlib import Path
import pandas as pd

In [11]:
key = "ed91a6e06d4a4e9a9d2b9419eba8a1f8"
textbook = input("Select your desired textbook: ")
file_path = Path(f"Textbooks/{textbook}.pdf")

In [12]:
reader = PdfReader(file_path)
raw_text = ''
total_pages = len(reader.pages)

In [13]:
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
texts = text_splitter.split_text(raw_text)

In [14]:
embeddings = OpenAIEmbeddings(
    chunk_size=1,
    openai_api_key=key,
    openai_api_type="azure",
    openai_api_base="https://codx-genai-server.openai.azure.com/",
    openai_api_version="2023-03-15-preview",
    deployment="embeddings-ada",
    max_retries=10
)
docsearch = FAISS.from_texts(texts, embeddings)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Get a vector representation of a given input that can be easily consumed by machine learning models and algorithms. Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Get a vector representation of a given input that can be easily consumed by machine learning models and algorithms. Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 4 seconds. Please go here: https://aka.ms/oai/quotaincrease if you

In [15]:
# Save docsearch to disk
docsearch.save_local(f"Embeddings/{textbook}.faiss")